In [30]:
import pandas as pd
import numpy as np
import sys
import os
import gc
from loguru import logger

sys.path.append("../../../../note")
sys.path.append(os.getcwd())

%load_ext autoreload
%autoreload 2

from module.get_info_FinMind import FinMindClient
from module.get_info_Postgre import PostgreClient
pg = PostgreClient('windows', database='stock_daily')
from analyzer import TXAnalyzer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# get data

In [31]:
START = "2020-01-01"
END = "2026-01-20"

fm = FinMindClient()
fm.initialize_frame(stock_id="TX", start_time=START, end_time=END)
analyzer = TXAnalyzer(fm.get_future_price())
margin_df = fm.get_total_margin_info()
margin_df.to_csv('../../../data/margin_info.csv')

margin_maintenance = fm.get_total_margin_maintenance(start_time=START, end_time=END)
margin_info = pg.fetch_table('total_margin_info')

2026-01-22 12:47:39.119 | INFO     | FinMind.data.finmind_api:get_data:171 - download Dataset.TaiwanFuturesDaily, data_id: TX
2026-01-22 12:47:47.750 | INFO     | FinMind.data.finmind_api:get_data:171 - download Dataset.TaiwanStockTotalMarginPurchaseShortSale, data_id: 
2026-01-22 12:47:48.128 | INFO     | FinMind.data.finmind_api:get_data:171 - download Dataset.TaiwanTotalExchangeMarginMaintenance, data_id: 


# analysis

## 日盤跟夜盤分開的 dilay ret & 月報酬
### `漲都是夜盤在漲，日盤上上下下不太明顯`
### 但 2、4 月可以日盤當沖

In [66]:
analyzer.daily_ret()
analyzer.monthly_ret()
analyzer.monthly_ret(session='day')

## `把日盤 ret 當指標，看夜盤反映`
看起來極端情況都會跌

In [57]:
analyzer.indicator_position_ret()

## `牛熊判斷`
divergence > 0 後的確好一點點，但比日本壓縮機還稀少

In [27]:
analyzer.indicator_bull_or_bear(demean=False)

count    902.000000
mean       0.000573
std        0.006094
min       -0.027879
25%       -0.002781
50%        0.000540
75%        0.004146
max        0.037598
Name: daily_ret_a, dtype: float64

count    901.000000
mean       0.000504
std        0.006670
min       -0.030721
25%       -0.003616
50%        0.000505
75%        0.004699
max        0.020629
Name: daily_ret, dtype: float64

In [26]:
analyzer.indicator_bull_or_bear(demean=True)

count    902.000000
mean       0.000018
std        0.006094
min       -0.028434
25%       -0.003336
50%       -0.000015
75%        0.003591
max        0.037043
Name: daily_ret_a, dtype: float64

count    901.000000
mean       0.000386
std        0.006670
min       -0.030838
25%       -0.003733
50%        0.000387
75%        0.004581
max        0.020511
Name: daily_ret, dtype: float64

## `放假天數`

In [34]:
analyzer.indicator_gap_days(demean=False, point_version=False)

In [33]:
analyzer.indicator_gap_days(demean=True, point_version=False)

## `大盤融資維持率`
適合超底??

大於 160 就穩定下降，135 ~ 155 上升  
但每個券商、資料商的資料算法不一定一樣

In [34]:
temp_df = analyzer.display_df()
temp_df = temp_df.reset_index(names='date')
margin_maintenance['date'] = pd.to_datetime(margin_maintenance['date'])
temp_df = temp_df.merge(margin_maintenance, how='left', on='date')
temp_df.set_index('date', inplace=True)
analyzer.update_df(temp_df)

,futures_id,contract_date,monthly_group,Open,High,Low,Close,spread,spread_per,Volume,...,spread_per_a,Volume_a,settlement_price_a,open_interest_a,trading_session_a,daily_ret,cum_daily_ret,daily_ret_a,cum_daily_ret_a,TotalExchangeMarginMaintenance
date,,,,,,,,,,,,,,,,,,,,,
2020-01-02,TX,202001,2020-01-15,12044.0,12120.0,12023.0,12102.0,108.0,0.90,100401.0,...,0.25,22506.0,0.0,0.0,after_market,0.004816,0.004816,0.002334,0.002334,166.764
2020-01-03,TX,202001,2020-01-15,12180.0,12198.0,11996.0,12086.0,-15.0,-0.12,172660.0,...,0.59,32214.0,0.0,0.0,after_market,-0.007718,-0.002902,0.006034,0.008368,165.209
2020-01-06,TX,202001,2020-01-15,12017.0,12034.0,11948.0,11950.0,-137.0,-1.13,118380.0,...,-0.36,36868.0,0.0,0.0,after_market,-0.005575,-0.008477,-0.004052,0.004315,163.083
2020-01-07,TX,202001,2020-01-15,11995.0,12009.0,11816.0,11871.0,-78.0,-0.65,157264.0,...,0.23,38896.0,0.0,0.0,after_market,-0.010338,-0.018815,0.003183,0.007499,160.375
2020-01-08,TX,202001,2020-01-15,11728.0,11892.0,11697.0,11788.0,-83.0,-0.70,158971.0,...,0.31,36974.0,0.0,0.0,after_market,0.005116,-0.013699,0.003455,0.010953,159.042
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026-01-14,TX,202601,2026-01-21,31000.0,31162.0,30937.0,30982.0,112.0,0.36,54974.0,...,0.28,52649.0,0.0,0.0,after_market,-0.000581,0.086057,0.002786,0.788606,174.515
2026-01-15,TX,202601,2026-01-21,30831.0,31057.0,30744.0,31050.0,70.0,0.23,58154.0,...,-0.45,52310.0,0.0,0.0,after_market,0.007103,0.093160,-0.003200,0.785407,173.912
2026-01-16,TX,202601,2026-01-21,31382.0,31536.0,31157.0,31474.0,424.0,1.37,72871.0,...,0.92,65948.0,0.0,0.0,after_market,0.002932,0.096092,0.004907,0.790313,176.183


In [52]:
analyzer.indicator_maintenance_rate(demean=False, point_version=False)

In [51]:
analyzer.indicator_maintenance_rate(demean=True, point_version=False)

## 融資

In [17]:
temp_df = analyzer.display_df()
temp_df = temp_df.reset_index(names='date')
margin_tf = margin_df.pivot_table(index='date', columns='name', values='TodayBalance')
margin_tf.reset_index(inplace=True)
margin_tf['date'] = pd.to_datetime(margin_tf['date'])
temp_df = temp_df.merge(margin_tf, how='left', on='date')
temp_df.set_index('date', inplace=True)
analyzer.update_df(temp_df)
analyzer.display_df()

,futures_id,contract_date,monthly_group,Open,High,Low,Close,spread,spread_per,Volume,...,settlement_price_a,open_interest_a,trading_session_a,daily_ret,cum_daily_ret,daily_ret_a,cum_daily_ret_a,MarginPurchase,MarginPurchaseMoney,ShortSale
date,,,,,,,,,,,,,,,,,,,,,
2020-01-02,TX,202001,2020-01-15,12044.0,12120.0,12023.0,12102.0,108.0,0.90,100401.0,...,0.0,0.0,after_market,0.004816,0.004816,0.002334,0.002334,7344427.0,1.456043e+11,654584.0
2020-01-03,TX,202001,2020-01-15,12180.0,12198.0,11996.0,12086.0,-15.0,-0.12,172660.0,...,0.0,0.0,after_market,-0.007718,-0.002902,0.006034,0.008368,7322773.0,1.446173e+11,652415.0
2020-01-06,TX,202001,2020-01-15,12017.0,12034.0,11948.0,11950.0,-137.0,-1.13,118380.0,...,0.0,0.0,after_market,-0.005575,-0.008477,-0.004052,0.004315,7338109.0,1.454611e+11,690772.0
2020-01-07,TX,202001,2020-01-15,11995.0,12009.0,11816.0,11871.0,-78.0,-0.65,157264.0,...,0.0,0.0,after_market,-0.010338,-0.018815,0.003183,0.007499,7314998.0,1.445947e+11,691277.0
2020-01-08,TX,202001,2020-01-15,11728.0,11892.0,11697.0,11788.0,-83.0,-0.70,158971.0,...,0.0,0.0,after_market,0.005116,-0.013699,0.003455,0.010953,7297808.0,1.436928e+11,662743.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026-01-14,TX,202601,2026-01-21,31000.0,31162.0,30937.0,30982.0,112.0,0.36,54974.0,...,0.0,0.0,after_market,-0.000581,0.086057,0.002786,0.788606,8040168.0,3.584528e+11,323345.0
2026-01-15,TX,202601,2026-01-21,30831.0,31057.0,30744.0,31050.0,70.0,0.23,58154.0,...,0.0,0.0,after_market,0.007103,0.093160,-0.003200,0.785407,8045393.0,3.592992e+11,337855.0
2026-01-16,TX,202601,2026-01-21,31382.0,31536.0,31157.0,31474.0,424.0,1.37,72871.0,...,0.0,0.0,after_market,0.002932,0.096092,0.004907,0.790313,8129811.0,3.633918e+11,321614.0


In [61]:
analyzer.indicator_margin_delta(demean=True)

## 三大法人買賣

In [45]:
df_inst = pd.read_csv('../../../data/整體市場三大法人買賣表.csv')

df_inst['日期'] = pd.to_datetime(df_inst['日期'])
df_inst['Net'] = df_inst['買進'] - df_inst['賣出']
df_inst.rename(columns={'買進': 'buy', '賣出': 'sell'}, inplace=True)
pivot_df = df_inst.pivot(index='日期', columns='種類', values=['buy', 'sell', 'Net'])
pivot_df.columns = [f"{col[0]}_{col[1]}" for col in pivot_df.columns]

temp_df = analyzer.display_df()
temp_df = temp_df.merge(pivot_df, left_index=True, right_index=True, how='left')
analyzer.update_df(temp_df)

,futures_id,contract_date,monthly_group,Open,High,Low,Close,spread,spread_per,Volume,...,sell_Foreign_Dealer_Self,sell_Foreign_Investor,sell_Investment_Trust,sell_total,Net_Dealer_Hedging,Net_Dealer_self,Net_Foreign_Dealer_Self,Net_Foreign_Investor,Net_Investment_Trust,Net_total
date,,,,,,,,,,,,,,,,,,,,,
2020-01-02,TX,202001,2020-01-15,12044.0,12120.0,12023.0,12102.0,108.0,0.90,100401.0,...,12604610.0,2.825961e+10,2.184135e+09,3.615252e+10,2.376466e+09,1.395424e+09,-2752060.0,-3.429870e+09,-7.761760e+07,2.644024e+08
2020-01-03,TX,202001,2020-01-15,12180.0,12198.0,11996.0,12086.0,-15.0,-0.12,172660.0,...,12140490.0,3.747450e+10,2.398021e+09,5.089674e+10,-4.369021e+08,-5.680259e+08,3876970.0,-1.358661e+08,1.822129e+08,-9.585813e+08
2020-01-06,TX,202001,2020-01-15,12017.0,12034.0,11948.0,11950.0,-137.0,-1.13,118380.0,...,8498430.0,3.741593e+10,3.284215e+09,4.800463e+10,6.568383e+08,-9.574017e+08,865550.0,-9.750910e+09,-1.785311e+09,-1.183678e+10
2020-01-07,TX,202001,2020-01-15,11995.0,12009.0,11816.0,11871.0,-78.0,-0.65,157264.0,...,8676200.0,4.532241e+10,2.851857e+09,5.695957e+10,-7.147001e+08,-1.225375e+09,204180.0,-9.820883e+09,-1.219425e+09,-1.298038e+10
2020-01-08,TX,202001,2020-01-15,11728.0,11892.0,11697.0,11788.0,-83.0,-0.70,158971.0,...,7482080.0,3.842789e+10,1.583485e+09,4.820975e+10,-2.958244e+08,-5.279640e+08,1043530.0,-1.232386e+09,3.993776e+08,-1.656796e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026-01-14,TX,202601,2026-01-21,31000.0,31162.0,30937.0,30982.0,112.0,0.36,54974.0,...,0.0,2.143468e+11,1.738227e+10,2.605514e+11,6.188131e+09,9.249682e+08,0.0,1.328250e+10,-6.511675e+09,1.388393e+10
2026-01-15,TX,202601,2026-01-21,30831.0,31057.0,30744.0,31050.0,70.0,0.23,58154.0,...,0.0,2.265881e+11,1.510475e+10,2.756868e+11,-4.590572e+09,-1.252696e+09,0.0,-2.102245e+08,-2.779112e+09,-8.832604e+09
2026-01-16,TX,202601,2026-01-21,31382.0,31536.0,31157.0,31474.0,424.0,1.37,72871.0,...,0.0,2.398726e+11,1.758191e+10,2.903068e+11,6.032128e+09,-9.776937e+08,0.0,3.991656e+10,-7.719604e+09,3.725139e+10


In [62]:
analyzer.indicator_institutional_flow(demean=True)

## `美國公債`

### 長債
    yield_shock = US_bond_5y - US_bond_5y.shift(20)
yield_shock 超過 0.15 就不做多，甚至做空

    yield_divergence = (US_bond_5y / 30ma_5y) - 1
yield_divergence > 0.06 跌爛

    US_bond_5y 倒掛後快速回升要小心
yield_presure > 0.3 也跌爛

### 短債
    cash_crunch = US_bond_3m.diff(5)


In [41]:
bond_1_month_df = fm.get_US_bond('United States 1-Month', START, END)
bond_2_month_df = fm.get_US_bond('United States 2-Month', START, END)
bond_3_month_df = fm.get_US_bond('United States 3-Month', START, END)
bond_6_month_df = fm.get_US_bond('United States 6-Month', START, END)
bond_1_year_df = fm.get_US_bond('United States 1-Year', START, END)
bond_3_year_df = fm.get_US_bond('United States 3-Year', START, END)
bond_5_year_df = fm.get_US_bond('United States 5-Year', START, END)

temp_df = analyzer.display_df()
temp_df.reset_index(inplace=True)
temp_df['date'] = pd.to_datetime(temp_df['date'])

# Merge bonds with renaming to avoid duplicates
bond_map = {
    '1m': bond_1_month_df,
    '2m': bond_2_month_df,
    '3m': bond_3_month_df,
    '6m': bond_6_month_df,
    '1y': bond_1_year_df,
    '3y': bond_3_year_df,
    '5y': bond_5_year_df
}

for suffix, df in bond_map.items():
    # Extract only necessary columns and rename
    if 'value' in df.columns:
        sub_df = df[['date', 'value']].rename(columns={'value': f'US_bond_{suffix}'})
        temp_df = temp_df.merge(sub_df, on='date', how='left')
    else:
        # Fallback if columns are different, merge directly but be careful
        temp_df = temp_df.merge(df, on='date', how='left')

temp_df.set_index('date', inplace=True)

analyzer.update_df(temp_df)


2026-01-22 13:24:53.025 | INFO     | FinMind.data.finmind_api:get_data:171 - download Dataset.GovernmentBondsYield, data_id: United States 1-Month
2026-01-22 13:24:53.273 | INFO     | FinMind.data.finmind_api:get_data:171 - download Dataset.GovernmentBondsYield, data_id: United States 2-Month
2026-01-22 13:24:53.456 | INFO     | FinMind.data.finmind_api:get_data:171 - download Dataset.GovernmentBondsYield, data_id: United States 3-Month
2026-01-22 13:24:53.572 | INFO     | FinMind.data.finmind_api:get_data:171 - download Dataset.GovernmentBondsYield, data_id: United States 6-Month
2026-01-22 13:24:53.678 | INFO     | FinMind.data.finmind_api:get_data:171 - download Dataset.GovernmentBondsYield, data_id: United States 1-Year
2026-01-22 13:24:53.784 | INFO     | FinMind.data.finmind_api:get_data:171 - download Dataset.GovernmentBondsYield, data_id: United States 3-Year
2026-01-22 13:24:53.893 | INFO     | FinMind.data.finmind_api:get_data:171 - download Dataset.GovernmentBondsYield, data

,level_0,index,futures_id,contract_date,monthly_group,Open,High,Low,Close,spread,...,daily_ret_a,cum_daily_ret_a,TotalExchangeMarginMaintenance,US_bond_1m,US_bond_2m,US_bond_3m,US_bond_6m,US_bond_1y,US_bond_3y,US_bond_5y
date,,,,,,,,,,,,,,,,,,,,,
2020-01-02,0,0,TX,202001,2020-01-15,12044.0,12120.0,12023.0,12102.0,108.0,...,0.002334,0.002334,166.764,1.53,1.55,1.54,1.57,1.56,1.59,1.67
2020-01-03,1,1,TX,202001,2020-01-15,12180.0,12198.0,11996.0,12086.0,-15.0,...,0.006034,0.008368,165.209,1.52,1.55,1.52,1.55,1.55,1.54,1.59
2020-01-06,2,2,TX,202001,2020-01-15,12017.0,12034.0,11948.0,11950.0,-137.0,...,-0.004052,0.004315,163.083,1.54,1.54,1.56,1.56,1.54,1.56,1.61
2020-01-07,3,3,TX,202001,2020-01-15,11995.0,12009.0,11816.0,11871.0,-78.0,...,0.003183,0.007499,160.375,1.52,1.53,1.54,1.56,1.53,1.55,1.62
2020-01-08,4,4,TX,202001,2020-01-15,11728.0,11892.0,11697.0,11788.0,-83.0,...,0.003455,0.010953,159.042,1.50,1.53,1.54,1.56,1.55,1.61,1.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026-01-14,1473,1473,TX,202601,2026-01-21,31000.0,31162.0,30937.0,30982.0,112.0,...,0.002786,0.788606,174.515,3.72,3.66,3.67,3.58,3.50,3.56,3.72
2026-01-15,1474,1474,TX,202601,2026-01-21,30831.0,31057.0,30744.0,31050.0,70.0,...,-0.003200,0.785407,173.912,3.75,3.69,3.68,3.60,3.54,3.62,3.77
2026-01-16,1475,1475,TX,202601,2026-01-21,31382.0,31536.0,31157.0,31474.0,424.0,...,0.004907,0.790313,176.183,3.75,3.68,3.67,3.60,3.55,3.67,3.82


### long term

In [74]:
analyzer.indicator_US_bond(demean=True, indicator='yield_shock')
analyzer.indicator_US_bond(demean=True, indicator='yield_divergence')
analyzer.indicator_US_bond(demean=True, indicator='yield_presure')

### short term

In [75]:
analyzer.indicator_US_bond(demean=False, indicator='cash_crunch')
analyzer.indicator_US_bond(demean=True, indicator='near_inversion')
analyzer.indicator_US_bond(demean=True, indicator='near_yield_vol')